In [1]:
import sys
import os

import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from pathlib import Path
import ot
import scipy as sp
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from dsc180a_q1_project.utils import LoadCloudPoint, DistanceProfile

In [2]:
import random

random.seed(10)

# Set up project root and CSV path
current_dir = Path.cwd()
project_root = current_dir.parent
sys.path.insert(0, str(project_root))
csv_path = project_root / "datasets/csv_files/0005_Jogging001.csv"

lcp = LoadCloudPoint(filepath=csv_path)
source_pc, target_pc = lcp.get_two_random_point_cloud()

pc = lcp.get_entire_point_cloud()

dp = DistanceProfile(source_pc, target_pc)
distance_matrix = dp.compute_L2_matrix()

Loaded point cloud data from /Users/cadenp/Documents/GitHub/DSC180A-Q1-Project/datasets/csv_files/0005_Jogging001.csv, number of frames: 1377


In [4]:
df = pd.read_csv(csv_path)
df.head()

,M1_X,M1_Y,M1_Z,M2_X,M2_Y,M2_Z,M3_X,M3_Y,M3_Z,M4_X,...,M50_Z,M51_X,M51_Y,M51_Z,M52_X,M52_Y,M52_Z,M53_X,M53_Y,M53_Z
0,2639.408,1623.648,-70.160,2562.207,1553.125,-54.364,2557.960,1561.032,61.988,2728.980,...,297.047,2920.688,80.724,127.611,2850.494,72.129,102.185,2888.599,61.755,66.186
1,2643.200,1627.623,-81.553,2566.004,1557.344,-65.609,2562.197,1565.164,50.817,2732.780,...,268.192,2926.928,81.598,93.677,2854.581,73.878,69.548,2892.118,65.277,32.341
2,2647.051,1631.207,-92.894,2569.835,1561.324,-77.109,2566.441,1568.642,39.689,2736.640,...,238.676,2930.494,82.151,60.602,2857.725,76.385,37.374,2894.553,69.526,-1.013
3,2650.779,1634.278,-104.023,2573.612,1564.526,-88.228,2570.636,1571.503,28.615,2740.441,...,209.250,2933.272,83.310,27.928,2860.030,79.221,5.292,2895.423,74.199,-33.798
4,2654.546,1636.884,-115.159,2577.346,1567.232,-99.153,2574.831,1573.770,17.600,2744.226,...,180.167,2935.326,84.585,-3.779,2861.536,82.725,-25.269,2896.190,79.600,-65.216


In [7]:
pc.shape

(1377, 53, 3)

In [9]:
df = pd.DataFrame(pc.reshape(1377, 53, 3).tolist())
df = df.map(lambda x: np.nan if x[1] == 0 and x[2] == 0 else x)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,43,44,45,46,47,48,49,50,51,52
0,"[2639.408, 1623.648, -70.16]","[2562.207, 1553.125, -54.364]","[2557.96, 1561.032, 61.988]","[2728.98, 1563.85, -40.189]","[2715.151, 1562.895, 66.493]","[2633.81, 1400.657, 161.775]","[2631.723, 1184.764, 234.021]","[2692.918, 1299.043, 14.575]","[2714.223, 1133.353, 5.638]","[2568.54, 1288.18, 2.951]",...,"[2697.756, 39.341, 281.603]","[2864.603, 638.126, 159.498]","[2884.357, 484.882, 56.574]","[2791.747, 462.429, 13.453]","[2878.136, 302.407, 108.465]","[2863.459, 144.102, 234.692]","[2791.715, 117.01, 297.047]","[2920.688, 80.724, 127.611]","[2850.494, 72.129, 102.185]","[2888.599, 61.755, 66.186]"
1,"[2643.2, 1627.623, -81.553]","[2566.004, 1557.344, -65.609]","[2562.197, 1565.164, 50.817]","[2732.78, 1567.653, -52.011]","[2719.448, 1566.692, 54.699]","[2637.846, 1404.743, 149.644]","[2635.969, 1189.142, 221.697]","[2696.818, 1303.71, 2.438]","[2717.802, 1138.007, -6.784]","[2572.596, 1293.201, -9.466]",...,"[2697.641, 39.175, 280.276]","[2866.871, 642.389, 147.187]","[2885.786, 488.758, 43.926]","[2793.468, 467.083, 1.01]","[2881.931, 304.022, 87.422]","[2870.017, 139.111, 205.276]","[2799.815, 108.892, 268.192]","[2926.928, 81.598, 93.677]","[2854.581, 73.878, 69.548]","[2892.118, 65.277, 32.341]"
2,"[2647.051, 1631.207, -92.894]","[2569.835, 1561.324, -77.109]","[2566.441, 1568.642, 39.689]","[2736.64, 1571.022, -63.792]","[2723.671, 1569.912, 42.958]","[2641.871, 1408.782, 138.406]","[2640.165, 1193.003, 209.387]","[2700.742, 1307.852, -9.937]","[2721.377, 1142.09, -19.272]","[2576.564, 1297.688, -21.762]",...,"[2697.726, 38.591, 279.258]","[2869.217, 646.174, 135.176]","[2887.33, 492.025, 31.467]","[2795.183, 471.141, -11.043]","[2885.444, 305.427, 66.572]","[2876.041, 134.658, 175.89]","[2807.084, 101.078, 238.676]","[2930.494, 82.151, 60.602]","[2857.725, 76.385, 37.374]","[2894.553, 69.526, -1.013]"
3,"[2650.779, 1634.278, -104.023]","[2573.612, 1564.526, -88.228]","[2570.636, 1571.503, 28.615]","[2740.441, 1573.896, -75.51]","[2727.887, 1572.542, 31.25]","[2645.966, 1411.779, 126.234]","[2644.283, 1196.265, 196.993]","[2704.6, 1311.334, -22.283]","[2724.812, 1145.516, -31.786]","[2580.44, 1301.571, -34.074]",...,"[2697.259, 38.43, 277.4]","[2871.8, 649.478, 123.336]","[2889.045, 494.612, 19.195]","[2796.926, 474.584, -22.674]","[2888.67, 306.592, 45.989]","[2881.385, 130.303, 146.871]","[2813.491, 93.735, 209.25]","[2933.272, 83.31, 27.928]","[2860.03, 79.221, 5.292]","[2895.423, 74.199, -33.798]"
4,"[2654.546, 1636.884, -115.159]","[2577.346, 1567.232, -99.153]","[2574.831, 1573.77, 17.6]","[2744.226, 1576.302, -87.198]","[2732.053, 1574.548, 19.587]","[2650.114, 1414.174, 114.057]","[2648.404, 1198.901, 184.789]","[2708.348, 1314.013, -34.616]","[2728.238, 1148.238, -44.326]","[2584.24, 1304.723, -46.252]",...,"[2696.528, 38.428, 274.142]","[2874.842, 652.229, 111.488]","[2890.928, 496.298, 7.133]","[2798.783, 477.313, -34.068]","[2891.547, 307.431, 25.72]","[2886.001, 126.349, 118.345]","[2819.162, 86.862, 180.167]","[2935.326, 84.585, -3.779]","[2861.536, 82.725, -25.269]","[2896.19, 79.6, -65.216]"


In [10]:
df.isna().sum(axis=0)

0       0
1       0
2       0
3       8
4       0
5       0
6       0
7      19
8     190
9       1
10      0
11      0
12      0
13     44
14      3
15    100
16      1
17      0
18      0
19     23
20      1
21      0
22      8
23     97
24    184
25     58
26     13
27     72
28     30
29    268
30    127
31      0
32    370
33      7
34      3
35      0
36      0
37    241
38      0
39      0
40      1
41      0
42     47
43      4
44      2
45     12
46     93
47      7
48     38
49      0
50     32
51      3
52      8
dtype: int64

In [11]:
first = lcp.get_point_cloud_at_index(161)
df1 = pd.DataFrame(first, columns=["x", "y", "z"])
df1['index'] = df1.index
fig = px.scatter_3d(df1, x="x", y="y", z="z", hover_name="index",color='index',color_continuous_scale=px.colors.qualitative.Plotly)
fig.update_layout(height=600, width=800)

fig.show()